## HERMES - furosemide phenotype

#### Clone HeRmes repository

In [ ]:
#system("git clone https://github.com/nicksunderland/heRmes.git")

#### Pushing changes to Github

In [ ]:
# in ther terminal run
# cd /opt/notebooks
# dx download git_push.sh
# run bash git_push.sh 

#### Project & record ID

In [1]:
projectid <- "project-GvZyZ20J81vgPJGbJy8pgpyq"
recordid  <- "record-Gvb0Bg0Jfxfv0q8Fb2pXqKjg"

#### Libraries

In [2]:
library(glue)
library(data.table)
library(yaml)
source("/opt/notebooks/heRmes/R/ukbb_extraction_utils.R")

## Extract data
Given the large datasets we make use of the `dx run table-exporter` to extract the required phenotype data. The extraction function below will create a table-exportor job which you will be able to track in the 'Monitor' table on your RAP's homepage. The data will be extracted to your project into a folder called `hermes3_data`. The data is not immediately uploaded to this session, although we will import it later. 

To get the small data dictionaries locally in this session we use the `dx extract_dataset` function. 

#### Download data dictionary

In [3]:
setwd("/opt/notebooks")
dataset <- glue("{projectid}:{recordid}")
cmd <- glue("dx extract_dataset {dataset} -ddd")
system(cmd)
dict_files <- list.files(pattern="codings|data_dictionary|entity_dictionary")
data_dict_file <- dict_files[grepl("data_dictionary", dict_files)]

#### Read the extraction config file

In [9]:
config <- read_yaml("/opt/notebooks/heRmes/scripts/ukbb_extraction_config.yml")

extraction_template <- lapply(config, function(table) filter_ukbb_data_dict(data_dict_file, table$entity, table$columns))
                       
lapply(extraction_template, head, Inf)   # 3->Inf to see all            

item,entity,name,type,primary_key_type,coding_name,concept,description,folder_path,is_multi_select,is_sparse_coding,linkout,longitudinal_axis_type,referenced_entity_field,relationship,title,units
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>
eid,participant,eid,string,global,,NA,NA,Participant Information,,,,NA,,,Participant ID,
assessment_date,participant,p53_i0,date,,,NA,NA,Assessment centre > Recruitment > Reception,,,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=53,NA,,,Date of attending assessment centre | Instance 0,
assessment_date,participant,p53_i1,date,,,NA,NA,Assessment centre > Recruitment > Reception,,,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=53,NA,,,Date of attending assessment centre | Instance 1,
assessment_date,participant,p53_i2,date,,,NA,NA,Assessment centre > Recruitment > Reception,,,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=53,NA,,,Date of attending assessment centre | Instance 2,
assessment_date,participant,p53_i3,date,,,NA,NA,Assessment centre > Recruitment > Reception,,,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=53,NA,,,Date of attending assessment centre | Instance 3,
assessment_age,participant,p21003_i0,integer,,,NA,NA,Assessment centre > Recruitment > Reception,,,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=21003,NA,,,Age when attended assessment centre | Instance 0,years
assessment_age,participant,p21003_i1,integer,,,NA,NA,Assessment centre > Recruitment > Reception,,,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=21003,NA,,,Age when attended assessment centre | Instance 1,years
assessment_age,participant,p21003_i2,integer,,,NA,NA,Assessment centre > Recruitment > Reception,,,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=21003,NA,,,Age when attended assessment centre | Instance 2,years
assessment_age,participant,p21003_i3,integer,,,NA,NA,Assessment centre > Recruitment > Reception,,,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=21003,NA,,,Age when attended assessment centre | Instance 3,years


#### Run Table-Exporter extraction
Check the monitor tab in the project for completion of the file extraction jobs

In [10]:
for (table in names(config)) {
    
    filepath <- file.path("/mnt/project", config[[table]][["output"]]) 
    if (!file.exists(filepath)) {
        extract_ukbb_data(dataset = dataset, 
                          fields  = extraction_template[[table]][["name"]],
                          entity  = config[[table]][["entity"]],
                          output  = config[[table]][["output"]])
    } else {
        cat("File:", filepath, "- exists\n")
    }
    
}

File: /mnt/project/hermes3_data/data_participant.tsv - exists
File: /mnt/project/hermes3_data/data_hesin.tsv - exists
File: /mnt/project/hermes3_data/data_hesin_diag.tsv - exists
File: /mnt/project/hermes3_data/data_hesin_oper.tsv - exists
File: /mnt/project/hermes3_data/data_gp_clinical.tsv - exists
File: /mnt/project/hermes3_data/data_gp_scripts.tsv - exists
File: /mnt/project/hermes3_data/data_olink_instance_0.tsv - exists
File: /mnt/project/hermes3_data/data_olink_instance_2.tsv - exists
File: /mnt/project/hermes3_data/data_olink_instance_3.tsv - exists
